# Open Pose for keypoints extraction


## Installation

Run the code below to download and compile OpenPose within your Google Colab environment (based on [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks)). **This takes around 1 minute.** It is done when you see `[100%] Built target openpose_net`.

In [ ]:
import os
if not os.path.exists("openpose"):
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # !pip install -q youtube-dl
  !wget https://motionlab-storage.s3-us-west-2.amazonaws.com/openpose.tar.gz
  !tar -zxvf openpose.tar.gz
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  

Selecting previously unselected package libgflags2.2.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [ ]:
!cd content
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
!cd openpose/build && cmake .. && make -j`nproc`

/bin/bash: line 0: cd: content: No such file or directory
/bin/bash: line 0: cd: openpose/build: No such file or directory


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

     |████████████████████████████████| 1.9 MB 8.9 MB/s 
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Loo

##Mount Google Drive or upload your videos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Useful function to display a video

In [ ]:
# Define function to display the video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

# Display the video
# show_local_mp4_video('input.mp4', width=960, height=720)

#Run OpenPose

In [ ]:
import os
from pathlib import Path
from os.path import exists, join, basename, splitext

google_drive_path = '/content/drive/MyDrive'
folder_path = google_drive_path + '/faces_0'
# folder_path = '/content/videos'
# folders = os.listdir(folder_path)
print(os.path.isdir(folder_path))
folders = [ item for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item)) ]


result_folder = '/content/drive/MyDrive/BIWI_openpose'
if not os.path.isdir(result_folder):
  os.mkdir(result_folder)

list_to_be_done = []

for folder in folders:
  kpts_output_folder = os.path.join(result_folder, folder)
  try:
    os.mkdir(kpts_output_folder) # folder is just the name
  except(FileExistsError):
    pass
  folder = os.path.join(folder_path, folder) # folder is the entire path
  
  for filename in os.listdir(folder):
    current_file = os.path.join(folder, filename)

    if os.path.splitext(current_file)[-1] == '.png':
        print(current_file)
        list_to_be_done.append(current_file)
        # print(list_to_be_done)
        !cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 3 --video "$current_file" --write_json "$kpts_output_folder" --display 0 --render_pose 0



Streaming output truncated to the last 5000 lines.
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 3.050636 seconds.
/content/drive/MyDrive/faces_0/04/frame_00146_rgb.png
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 3.063976 seconds.
/content/drive/MyDrive/faces_0/04/frame_00137_rgb.png
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 3.017077 seconds.
/content/drive/MyDrive/faces_0/04/frame_00182_rgb.png
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPo

In [ ]:
!zip -r /content/keypoints.zip /content/drive/MyDrive/BIWI_openpose

  adding: content/drive/MyDrive/BIWI_openpose/ (stored 0%)
  adding: content/drive/MyDrive/BIWI_openpose/07/ (stored 0%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00425_rgb_000000000000_keypoints.json (deflated 61%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00450_rgb_000000000000_keypoints.json (deflated 61%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00410_rgb_000000000000_keypoints.json (deflated 62%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00417_rgb_000000000000_keypoints.json (deflated 61%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00414_rgb_000000000000_keypoints.json (deflated 61%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00406_rgb_000000000000_keypoints.json (deflated 61%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00438_rgb_000000000000_keypoints.json (deflated 61%)
  adding: content/drive/MyDrive/BIWI_openpose/07/frame_00420_rgb_000000000000_keypoints.json (deflated 61%)
  adding: conte

In [ ]:
from google.colab import files
files.download("/content/keypoints.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from os.path import exists, join, basename, splitext

# google_drive_path = '/content/drive/MyDrive'
# folder_path = google_drive_path + '/LAEO_OP/test'
folder_path = '/content/videos'
files = os.listdir(folder_path)
files.reverse()
for filename in files:
  if filename.endswith('.mp4'):
    # print(filename)
    colab_video_path = folder_path +'/'+ filename
    # print(colab_video_path)
    os.chmod(colab_video_path, 0o775)
    # create keypoints directory
    if os.path.exists(folder_path + '/keypoints') == False:
      os.mkdir(folder_path + '/keypoints')
    # create video directory for keypoints in json files
    colab_openpose_keypoints_path = folder_path + '/keypoints/' + filename.replace('.mp4', '') +'/'
    if os.path.exists(colab_openpose_keypoints_path) == False:
      os.mkdir(colab_openpose_keypoints_path)
    # print(colab_openpose_keypoints_path)
    if exists(colab_openpose_keypoints_path):
      # detect poses in these video frames using OpenPose
      !cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 10 --video "$colab_video_path" --write_json "$colab_openpose_keypoints_path" --display 0 --render_pose 0
      # !cd openpose && ./build/examples/openpose/openpose.bin --hand --face --number_people_max 12 --video '{colab_video_path}' --display 0 --write_video_with_audio --write_video '{colab_openpose_video_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25
      # !cd openpose && ./build/examples/openpose/openpose.bin --video '{colab_video_path}' --write_json '{output_path}' --display 0 --write_video ../openpose.avi

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 17.143104 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
Empty frame detected, frame number 189 of 192. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 189 of 192. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 189 of 192. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
OpenPose demo successfully finished. Total time: 15.420745 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them startin

zip folder to make 'downloadable'

In [ ]:
!zip -r /content/keypoints.zip /content/videos/keypoints

Streaming output truncated to the last 5000 lines.
  adding: content/videos/keypoints/Twd03/Twd03_000000000025_keypoints.json (deflated 62%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000118_keypoints.json (deflated 65%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000213_keypoints.json (deflated 64%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000060_keypoints.json (deflated 65%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000090_keypoints.json (deflated 66%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000245_keypoints.json (deflated 66%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000196_keypoints.json (deflated 64%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000268_keypoints.json (deflated 64%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000215_keypoints.json (deflated 64%)
  adding: content/videos/keypoints/Twd03/Twd03_000000000072_keypoints.json (deflated 68%)
  adding: content/videos/keypoints/Twd03/Twd03_00

In [ ]:
from os.path import exists, join, basename, splitext

# google_drive_path = '/content/drive/MyDrive'
# folder_path = google_drive_path + '/LAEO_OP/test'
folder_path = '/content/videos1'
files = os.listdir(folder_path)
files.reverse()
for filename in files:
  if filename.endswith('.avi'):
    # print(filename)
    colab_video_path = folder_path +'/'+ filename
    # print(colab_video_path)
    os.chmod(colab_video_path, 0o775)
    # create keypoints directory
    if os.path.exists(folder_path + '/keypoints') == False:
      os.mkdir(folder_path + '/keypoints')
    # create video directory for keypoints in json files
    colab_openpose_keypoints_path = folder_path + '/keypoints/' + filename.replace('.avi', '') +'/'
    if os.path.exists(colab_openpose_keypoints_path) == False:
      os.mkdir(colab_openpose_keypoints_path)
    # print(colab_openpose_keypoints_path)
    if exists(colab_openpose_keypoints_path):
      # detect poses in these video frames using OpenPose
      !cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 10 --video "$colab_video_path" --write_json "$colab_openpose_keypoints_path" --display 0 --render_pose 0
      # !cd openpose && ./build/examples/openpose/openpose.bin --hand --face --number_people_max 12 --video '{colab_video_path}' --display 0 --write_video_with_audio --write_video '{colab_openpose_video_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25
      # !cd openpose && ./build/examples/openpose/openpose.bin --video '{colab_video_path}' --write_json '{output_path}' --display 0 --write_video ../openpose.avi

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Unexpected end of file while searching for idx1 chunk
Failed to parse avi: index was not found
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
^C
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
^C
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
^C
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 51.842094 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo

In [ ]:
!zip -r /content/keypoints1.zip /content/videos1/keypoints

  adding: content/videos1/keypoints/ (stored 0%)
  adding: content/videos1/keypoints/si_alza_ed_esce/ (stored 0%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000041_keypoints.json (deflated 51%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000146_keypoints.json (stored 0%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000118_keypoints.json (stored 0%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000171_keypoints.json (deflated 63%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000155_keypoints.json (deflated 55%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000048_keypoints.json (deflated 61%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000024_keypoints.json (deflated 51%)
  adding: content/videos1/keypoints/si_alza_ed_esce/si_alza_ed_esce_000000000108_keypoints.json (deflated 55%)
  ad